In [3]:
import tensorflow as tf

In [29]:
import librosa
import numpy as np
import tensorflow 
import matplotlib
from scipy.signal import firwin, lfilter

In [5]:
# Load file âm thanh
def load_audio(file_path, sr=22050):
    audio, sr = librosa.load(file_path, sr=sr)
    return audio, sr

# Trích xuất MFCC
def extract_mfcc(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return mfccs

# # Ví dụ load và trích xuất đặc trưng
audio, sr = load_audio('C://Users/Admin/Downloads/file_example_WAV_10MG.wav')
mfcc_features = extract_mfcc(audio, sr)


In [6]:
from scipy.signal import firwin, lfilter

# Áp dụng bộ lọc FIR dải thông
def apply_fir_filter(data, sr, lowcut=None, highcut=None, numtaps=101, window='hamming'):
    if lowcut and highcut:
        fir_coeff = firwin(numtaps, [lowcut, highcut], pass_zero=False, fs=sr, window=window)
    elif highcut:
        fir_coeff = firwin(numtaps, highcut, pass_zero=True, fs=sr, window=window)
    elif lowcut:
        fir_coeff = firwin(numtaps, lowcut, pass_zero=False, fs=sr, window=window)
    else:
        raise ValueError("Phải chỉ định ít nhất lowcut hoặc highcut")
    return lfilter(fir_coeff, 1.0, data)

# Ví dụ áp dụng bộ lọc cho giọng nói (100 - 3000 Hz)
filtered_audio = apply_fir_filter(audio, sr, lowcut=100, highcut=3000, numtaps=101, window='hamming')


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Thiết kế mô hình CNN bằng PyTorch
class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 lớp âm thanh

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

# Tạo mô hình
model = AudioCNN()

# Compile mô hình (PyTorch sử dụng cách huấn luyện trực tiếp thay vì compile như Keras)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [43]:
import os


In [ ]:
# Lấy danh sách tất cả các file audio trong thư mục
audio_dir = "E:\\dataset\\sound-data\\sound-data\\audio"
file_paths = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]

# Giả sử bạn có danh sách nhãn tương ứng (ví dụ tất cả các file đều có nhãn 0)
labels = [0 for _ in range(len(file_paths))]  # Thay thế nhãn phù hợp với dữ liệu của bạn

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, random_state=42)

# Tạo dataset và dataloader cho tập huấn luyện
train_dataset = AudioDataset(train_paths, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Tạo dataset và dataloader cho tập kiểm tra
test_dataset = AudioDataset(test_paths, test_labels)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Vòng lặp huấn luyện
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        # Reset gradient
        optimizer.zero_grad()
        # Forward
        outputs = model(inputs)
        # Tính loss
        loss = criterion(outputs, targets)
        # Backward
        loss.backward()
        # Cập nhật trọng số
        optimizer.step()
        # Cộng dồn loss
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

print("Huấn luyện hoàn tất!")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

# Dataset tùy chỉnh cho dữ liệu âm thanh
class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và trích xuất đặc trưng MFCC
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Thiết kế mô hình CNN bằng PyTorch
class AudioCNN2(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 10 * 107, 128)
        self.fc2 = nn.Linear(128, 4)  # 4 lớp âm thanh

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        print(f"Kích thước sau lớp pooling: {x.shape}")  # In kích thước tensor để kiểm tra
        x = x.view(x.size(0), -1)  # Điều chỉnh kích thước để phù hợp với fully connected layer
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x


# Tạo mô hình
model = AudioCNN()

# Compile mô hình (PyTorch sử dụng cách huấn luyện trực tiếp thay vì compile như Keras)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Lấy danh sách tất cả các file audio trong thư mục
audio_dir = r"E:\dataset\sound-data\sound-data\audio"
file_paths = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]

# Sử dụng một file duy nhất để thử nghiệm
file_paths = file_paths[:1]
labels = [0]  # Thay thế nhãn phù hợp với dữ liệu của bạn

# Tạo dataset và dataloader cho tập huấn luyện
single_dataset = AudioDataset(file_paths, labels)
single_loader = DataLoader(single_dataset, batch_size=1, shuffle=False)

# Vòng lặp dự đoán cho một file duy nhất
model.eval()
with torch.no_grad():
    for inputs, targets in single_loader:
        outputs = model(inputs)
        predicted = torch.argmax(outputs, dim=1)
        print(f"Predicted: {predicted.item()}, Actual: {targets.item()}")

print("Dự đoán hoàn tất!")

model 1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

# Dataset tùy chỉnh cho dữ liệu âm thanh
class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và trích xuất đặc trưng MFCC
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Thiết kế mô hình CNN bằng PyTorch
class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = None  # Sẽ được khởi tạo sau
        self.fc2 = nn.Linear(128, 4)  # 4 lớp âm thanh

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Điều chỉnh kích thước để phù hợp với fully connected layer
        # self.fc1 = nn.Linear(32 * 10 * 107, 128)
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128).to(x.device)  # Khởi tạo fc1 với kích thước phù hợp
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Bỏ softmax ở đây vì CrossEntropyLoss đã bao gồm
        return x

# Tạo mô hình
model = AudioCNN()

# Compile mô hình (PyTorch sử dụng cách huấn luyện trực tiếp thay vì compile như Keras)
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Giảm learning rate để huấn luyện chậm hơn
criterion = nn.CrossEntropyLoss()

# Lấy danh sách tất cả các file audio trong thư mục
audio_dir = r"E:\dataset\sound-data\sound-data\audio"
file_paths = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, [0 for _ in range(len(file_paths))], test_size=0.3, random_state=42)  # Tăng tỷ lệ dữ liệu kiểm tra

# Tạo dataset và dataloader cho tập huấn luyện
train_dataset = AudioDataset(train_paths, train_labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = AudioDataset(test_paths, test_labels)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Tải mô hình nếu đã có mô hình được lưu trước đó
if os.path.exists("best_audio_classification_model.pth"):
    model.load_state_dict(torch.load("best_audio_classification_model.pth"), strict=False)
    print("Đã tải mô hình từ file 'best_audio_classification_model.pth'")

# Huấn luyện mô hình
model.train()
epoch = 0
best_accuracy = 0.0
while True:  # Huấn luyện tới khi đạt độ chính xác mong muốn
    running_loss = 0.0
    for _ in range(5):  # Huấn luyện ít nhất 5 lần mỗi epoch
        for inputs, targets in train_loader:
            try:
                optimizer.zero_grad()
                # Forward
                outputs = model(inputs)
                # Tính loss
                loss = criterion(outputs, targets)
                # Backward
                loss.backward()
                # Cập nhật trọng số
                optimizer.step()
                # Cộng dồn loss
                running_loss += loss.item()
            except RuntimeError as e:
                print(f"Error during training: {e}")
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")
    
    # Kiểm tra độ chính xác trên tập kiểm tra
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            try:
                outputs = model(inputs)
                predicted = torch.argmax(outputs, dim=1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
            except RuntimeError as e:
                print(f"Error during evaluation: {e}")
    accuracy = 100 * correct / total
    print(f"Độ chính xác trên tập kiểm tra: {accuracy}%")
    model.train()
    
    # Lưu mô hình nếu đạt độ chính xác tốt hơn
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_audio_classification_model.pth")
        print("Đã lưu mô hình tốt nhất vào file 'best_audio_classification_model.pth'")
    
    if accuracy >= 100.0:
        break
    epoch += 1

print("Huấn luyện hoàn tất!")

# Lưu trạng thái tối ưu sau khi huấn luyện
torch.save(model.state_dict(), "final_audio_classification_model.pth")
print("Đã lưu mô hình cuối cùng vào file 'final_audio_classification_model.pth'")

# Đánh giá mô hình
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        try:
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        except RuntimeError as e:
            print(f"Error during evaluation: {e}")

print(f"Độ chính xác của mô hình: {100 * correct / total}%")

# Dự đoán cho một file duy nhất
single_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
model.eval()
with torch.no_grad():
    for inputs, targets in single_loader:
        try:
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            print(f"Predicted: {predicted.item()}, Actual: {targets.item()}")
        except RuntimeError as e:
            print(f"Error during prediction: {e}")

print("Dự đoán hoàn tất!")

# Tiếp tục huấn luyện từ mô hình đã lưu
model.train()
epoch = 0
while True:  # Tiếp tục huấn luyện tới khi đạt độ chính xác mong muốn
    running_loss = 0.0
    for _ in range(5):  # Huấn luyện ít nhất 5 lần mỗi epoch
        for inputs, targets in train_loader:
            try:
                optimizer.zero_grad()
                # Forward
                outputs = model(inputs)
                # Tính loss
                loss = criterion(outputs, targets)
                # Backward
                loss.backward()
                # Cập nhật trọng số
                optimizer.step()
                # Cộng dồn loss
                running_loss += loss.item()
            except RuntimeError as e:
                print(f"Error during training: {e}")
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")
    
    # Kiểm tra độ chính xác trên tập kiểm tra
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            try:
                outputs = model(inputs)
                predicted = torch.argmax(outputs, dim=1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
            except RuntimeError as e:
                print(f"Error during evaluation: {e}")
    accuracy = 100 * correct / total
    print(f"Độ chính xác trên tập kiểm tra: {accuracy}%")
    model.train()


model 2


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

# Dataset tùy chỉnh cho dữ liệu âm thanh
class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và giới hạn về 10 giây
        audio, sr = librosa.load(file_path, sr=None, duration=5)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)



    def __getitem2__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và trích xuất đặc trưng MFCC
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Thiết kế mô hình CNN bằng PyTorch với Dropout để giảm overfitting
class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)  # Thêm Dropout với tỷ lệ 0.5
        self.fc1 = None  # Sẽ được khởi tạo sau
        self.fc2 = nn.Linear(128, 4)  # 4 lớp âm thanh

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128).to(x.device)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Áp dụng dropout trước lớp cuối cùng
        x = self.fc2(x)
        return x

# Tạo mô hình
model = AudioCNN()

# Optimizer và hàm loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# Lấy danh sách file âm thanh và kiểm tra nhãn
audio_dir = r"E:\dataset\sound-data\sound-data\audio"
file_paths = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]
labels = [0 for _ in range(len(file_paths))]  # Cập nhật nhãn phù hợp nếu có

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=0.3, stratify=labels, random_state=42)

# Tạo dataset và dataloader cho tập huấn luyện và kiểm tra
train_dataset = AudioDataset(train_paths, train_labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = AudioDataset(test_paths, test_labels)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Tải mô hình nếu đã có mô hình được lưu trước đó
if os.path.exists("best_audio_classification_model.pth"):
    model.load_state_dict(torch.load("best_audio_classification_model.pth"), strict=False)
    print("Đã tải mô hình từ file 'best_audio_classification_model.pth'")

# Huấn luyện mô hình
model.train()
epoch = 0
best_accuracy = 0.0


# Đặt số lượng epoch cố định
num_epochs = 50

# Huấn luyện mô hình
model.train()
best_accuracy = 0.0
for epoch in range(num_epochs):
    running_loss = 0.0
    total_batches = len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Tổng số batch trong tập huấn luyện: {total_batches}")

    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # In đường dẫn của các file âm thanh trong batch hiện tại
        batch_file_paths = [train_paths[idx] for idx in range(i * train_loader.batch_size, min((i+1) * train_loader.batch_size, len(train_paths)))]
        print(f"Batch {i+1}/{total_batches} - Đang huấn luyện trên các file âm thanh: {batch_file_paths}")

        # In ra sau mỗi 10 batch
        if (i + 1) % 10 == 0:
            print(f"Batch {i+1}/{total_batches} - Loss trung bình: {running_loss / (i + 1)}")

    print(f"Epoch {epoch+1}, Loss trung bình: {running_loss / total_batches}")
    
    # Kiểm tra độ chính xác trên tập kiểm tra
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for j, (inputs, targets) in enumerate(test_loader):
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
            # In đường dẫn của các file âm thanh trong batch kiểm tra hiện tại
            batch_file_paths = [test_paths[idx] for idx in range(j * test_loader.batch_size, min((j+1) * test_loader.batch_size, len(test_paths)))]
            print(f"Batch kiểm tra {j+1}/{len(test_loader)} - Đang kiểm tra trên các file âm thanh: {batch_file_paths}")
            
    accuracy = 100 * correct / total
    print(f"Độ chính xác trên tập kiểm tra: {accuracy}%")
    model.train()
    
    # Lưu mô hình nếu đạt độ chính xác tốt hơn
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_audio_classification_model.pth")
        print("Đã lưu mô hình tốt nhất vào file 'best_audio_classification_model.pth'")
    
    # Nếu đạt độ chính xác tối đa thì kết thúc sớm
    if accuracy >= 100.0:
        print("Đạt độ chính xác tối đa. Kết thúc sớm quá trình huấn luyện.")
        break

print("Huấn luyện hoàn tất!")



# Đánh giá mô hình
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f"Độ chính xác của mô hình: {100 * correct / total}%")


In [ ]:
# Lưu mô hình sau khi huấn luyện
torch.save(model.state_dict(), "audio_classification_model.pth")
print("Đã lưu mô hình vào file 'audio_classification_model.pth'")


In [ ]:
# Tạo lại mô hình và tải trọng số đã huấn luyện
model = AudioCNN()
model.load_state_dict(torch.load("audio_classification_model.pth"))
model.eval()  # Chuyển sang chế độ đánh giá
print("Đã tải mô hình từ file 'audio_classification_model.pth'")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

# Dataset tuỳ chỉnh cho dữ liệu âm thanh
class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và trích xuất đặc trưng MFCC
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Thiết kế mô hình CNN bằng PyTorch
class AudioCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv_output_size = self._get_conv_output((1, 40, 216))
        self.fc1 = nn.Linear(self.conv_output_size, 128)
        self.fc2 = nn.Linear(128, num_classes)  # num_classes là số lớp âm thanh

    def _get_conv_output(self, shape):
        with torch.no_grad():
            x = torch.zeros(1, *shape)
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            return x.numel()

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Điều chỉnh kích thước để phù hợp với fully connected layer
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Bỏ softmax ở đây vì CrossEntropyLoss đã bao gồm
        return x

# Kiểm tra thiết bị GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Tạo mô hình
model = AudioCNN().to(device)

# Khởi tạo optimizer và loss function
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Giảm learning rate để huấn luyện chậm hơn
criterion = nn.CrossEntropyLoss()

# Lấy danh sách tất cả các file audio trong thư mục
audio_dir = r"E:\dataset\sound-data\sound-data\audio"
file_paths = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]
labels = [0 for _ in range(len(file_paths))]  # Cập nhật nhãn phù hợp cho từng file

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.3, random_state=42)  # Tăng tỷ lệ dữ liệu kiểm tra

# Tạo dataset và dataloader cho tập huấn luyện
train_dataset = AudioDataset(train_paths, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)  # Tăng batch size để tối ưu tốc độ huấn luyện
test_dataset = AudioDataset(test_paths, test_labels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Tải mô hình nếu đã có mô hình được lưu trước đó
if os.path.exists("best_audio_classification_model.pth"):
    model.load_state_dict(torch.load("best_audio_classification_model.pth", map_location=device))
    print("Đã tải mô hình từ file 'best_audio_classification_model.pth'")

# Huấn luyện mô hình
model.train()
epoch = 0
best_accuracy = 0.0
patience = 5  # Số epoch nhắc nhở trước khi dừng huấn luyện nếu không cải thiện
trigger_times = 0

for epoch in range(100):  # Giới hạn số lượng epoch
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        # Forward
        outputs = model(inputs)
        # Tính loss
        loss = criterion(outputs, targets)
        # Backward
        loss.backward()
        # Cập nhật trọng số
        optimizer.step()
        # Cộng dồn loss
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}")
    
    # Kiểm tra độ chính xác trên tập kiểm tra
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    accuracy = 100 * correct / total
    print(f"Độ chính xác trên tập kiểm tra: {accuracy:.2f}%")
    model.train()
    
    # Lưu mô hình nếu đạt độ chính xác tốt hơn
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_audio_classification_model.pth")
        print("Đã lưu mô hình tốt nhất vào file 'best_audio_classification_model.pth'")
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Độ chính xác không cải thiện, dừng huấn luyện.")
            break
    
    if accuracy >= 99.0:  # Giới hạn độ chính xác tối đa là 99%
        break

print("Huấn luyện hoàn tất!")

# Lưu trạng thái tối ưu sau khi huấn luyện
torch.save(model.state_dict(), "final_audio_classification_model.pth")
print("Đã lưu mô hình cuối cùng vào file 'final_audio_classification_model.pth'")

# Đánh giá mô hình
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f"Độ chính xác cuối cùng của mô hình: {100 * correct / total:.2f}%")

# Dự đoán trên tập dữ liệu mới
def predict_new_data(model, new_file_paths):
    model.eval()
    new_dataset = AudioDataset(new_file_paths, [0] * len(new_file_paths))
    new_loader = DataLoader(new_dataset, batch_size=1, shuffle=False)
    predictions = []
    with torch.no_grad():
        for inputs, _ in new_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            predictions.append(predicted.item())
    return predictions

# Ví dụ dự đoán
new_file_paths = [r"E:\dataset\sound-data\new-audio\audio1.wav", r"E:\dataset\sound-data\new-audio\audio2.wav"]
predictions = predict_new_data(model, new_file_paths)
for i, prediction in enumerate(predictions):
    print(f"File {new_file_paths[i]} được dự đoán là lớp: {prediction}")


In [9]:
# Dự đoán trên tập dữ liệu mới
def predict_new_data(model, new_file_paths):
    model.eval()
    new_dataset = AudioDataset(new_file_paths, [0] * len(new_file_paths))
    new_loader = DataLoader(new_dataset, batch_size=1, shuffle=False)
    predictions = []
    with torch.no_grad():
        for inputs, _ in new_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            predictions.append(predicted.item())
    return predictions

# Ví dụ dự đoán
new_file_paths = [r"E:\dataset\sound-data\new-audio\audio1.wav", r"E:\dataset\sound-data\new-audio\audio2.wav"]
predictions = predict_new_data(model, new_file_paths)
for i, prediction in enumerate(predictions):
    print(f"File {new_file_paths[i]} được dự đoán là lớp: {prediction}")

NameError: name 'device' is not defined

In [ ]:
if os.path.exists("best_audio_classification_model.pth"):
    os.remove("best_audio_classification_model.pth")
    print("Đã xóa mô hình đã lưu trước đó.")